<a href="https://colab.research.google.com/github/Amilcar28/Amilcar28/blob/main/Manipular_dados_Era5_API_6_parametros_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#Install package library
!pip install pydap
!pip install netCDF4
!pip install cftime
!pip install xarray
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy


  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.1.3 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.1.3 which is incompatible.
langchain 0.3.4 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.1.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.
py

In [38]:
import xarray as xr
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import cftime
import matplotlib.dates as mdates
#import libarchive
import pydot
import cartopy

In [39]:
## acessado os dados do Era5 de termperatura horária de 2m pelo URL
t2m = xr.open_dataset('http://apdrc.soest.hawaii.edu/dods/public_data/Reanalysis_Data/ERA5/hourly/2m_temperature')
pr = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/Surface_total_precipitation')
HR = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/Specific_humidity_1000mb')
u_wind = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/U_wind_component_10m')
v_wind = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/V_wind_component_10m')
sp = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/Surface_pressure')
blh = xr.open_dataset('http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/Boundary_layer_height') # altura da camada limite


/usr/local/lib/python3.10/dist-packages/xarray/coding/times.py:187: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)


In [40]:
# Converter as direcões do Ventos do ERA5
def wind_dir(u, v):
    """Compute the wind direction from u and v-components.
    """
    wdir = np.mod(180 + np.rad2deg(np.arctan2(v, u)), 360)
    return wdir

In [41]:
# Converter as Velocidade do Ventos do ERA5
def wind_speed(u, v):
    """Help you to compute the speed."""
    wspeed = np.sqrt(u * u + v * v)
    return wspeed


In [42]:
# Imprimido a variavel de temperatura ERA5
t2m

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    t2       (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [43]:
## Imprimindo a Precipitação total do ERA5
pr

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    tp       (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface Total precipitation
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [44]:
# Imprimido os dados de Humidade especifica ERA5
HR

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    q        (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Specific humidity on 1000 hPa
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [45]:
# Imprimido os dados de Velocidade do vento (v_wind) ERA5.
v_wind

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    v10      (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface 10 metre V wind component
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [46]:

# Imprimido os dados de direcção do vento (u_wind) ERA5.
u_wind

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    u10      (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface 10 metre U wind component
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [47]:
# Imprimido os dados de direcção do vento (surface Pressure) ERA5.
sp

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    sp       (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface Pressure
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [48]:
# Imprimido os dados de direcção do vento (Boundary Layer Height) ERA5.
blh

<xarray.Dataset> Size: 2TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    blh      (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Boundary layer height
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [49]:
# Agrupar todos os dados em uma unica tabela.
era5_data = xr.merge([t2m, pr, HR, u_wind, v_wind, sp, blh])
era5_data

<xarray.Dataset> Size: 12TB
Dimensions:  (time: 401040, lat: 721, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    t2       (time, lat, lon) float32 2TB ...
    tp       (time, lat, lon) float32 2TB ...
    q        (time, lat, lon) float32 2TB ...
    u10      (time, lat, lon) float32 2TB ...
    v10      (time, lat, lon) float32 2TB ...
    sp       (time, lat, lon) float32 2TB ...
    blh      (time, lat, lon) float32 2TB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [50]:
# Selecionar a coordenada de Angola
ang_data = era5_data.sel(lat=slice(-19 , -3), lon=slice(11 , 25))
ang_data

<xarray.Dataset> Size: 42GB
Dimensions:  (time: 401040, lat: 65, lon: 57)
Coordinates:
  * time     (time) datetime64[ns] 3MB 1979-01-01 ... 2024-09-30T23:00:00.000003
  * lat      (lat) float64 520B -19.0 -18.75 -18.5 -18.25 ... -3.5 -3.25 -3.0
  * lon      (lon) float64 456B 11.0 11.25 11.5 11.75 ... 24.25 24.5 24.75 25.0
Data variables:
    t2       (time, lat, lon) float32 6GB ...
    tp       (time, lat, lon) float32 6GB ...
    q        (time, lat, lon) float32 6GB ...
    u10      (time, lat, lon) float32 6GB ...
    v10      (time, lat, lon) float32 6GB ...
    sp       (time, lat, lon) float32 6GB ...
    blh      (time, lat, lon) float32 6GB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [51]:
# Selecionar um unico periodo de dados
ang_data_2021 = ang_data.sel(time=slice('2021-03-01' , '2021-03-30'))
ang_data_2021

<xarray.Dataset> Size: 75MB
Dimensions:  (time: 720, lat: 65, lon: 57)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
  * lat      (lat) float64 520B -19.0 -18.75 -18.5 -18.25 ... -3.5 -3.25 -3.0
  * lon      (lon) float64 456B 11.0 11.25 11.5 11.75 ... 24.25 24.5 24.75 25.0
Data variables:
    t2       (time, lat, lon) float32 11MB ...
    tp       (time, lat, lon) float32 11MB ...
    q        (time, lat, lon) float32 11MB ...
    u10      (time, lat, lon) float32 11MB ...
    v10      (time, lat, lon) float32 11MB ...
    sp       (time, lat, lon) float32 11MB ...
    blh      (time, lat, lon) float32 11MB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [52]:
# Extrair dados de unico ponto exemplo para extrair um ponto
Luanda = ang_data_2021.sel(lat=(-8.82), lon=(13.22), method=('nearest'))
Luanda

<xarray.Dataset> Size: 26kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    t2       (time) float32 3kB ...
    tp       (time) float32 3kB ...
    q        (time) float32 3kB ...
    u10      (time) float32 3kB ...
    v10      (time) float32 3kB ...
    sp       (time) float32 3kB ...
    blh      (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [53]:
# Abrir todos os dados em uma unica tabela
data_df = Luanda.to_dataframe()
data_df

,lat,lon,t2,tp,q,u10,v10,sp,blh
time,,,,,,,,,
2021-03-01 00:00:00.000000,-8.75,13.25,299.137085,0.0,0.017407,0.826431,3.026978,100531.632812,446.928650
2021-03-01 00:59:59.999997,-8.75,13.25,298.967529,0.0,0.017420,0.461060,3.130447,100489.984375,462.804993
2021-03-01 02:00:00.000003,-8.75,13.25,298.566162,0.0,0.017444,0.009598,2.948959,100461.578125,476.581573
2021-03-01 03:00:00.000000,-8.75,13.25,298.736145,0.0,0.017453,-0.408035,2.615021,100462.406250,437.047516
2021-03-01 03:59:59.999997,-8.75,13.25,298.585693,0.0,0.017451,-0.770386,2.416183,100504.570312,381.725128
...,...,...,...,...,...,...,...,...,...
2021-03-30 18:59:59.999997,-8.75,13.25,300.694122,0.0,0.017289,2.862839,0.835739,100690.843750,546.046326
2021-03-30 20:00:00.000003,-8.75,13.25,300.440125,0.0,0.017369,2.234467,1.203201,100794.406250,512.461609
2021-03-30 21:00:00.000000,-8.75,13.25,300.186157,0.0,0.017384,1.724518,1.453690,100832.953125,441.784576


In [23]:
# data frame
data_df.tail()

,lat,lon,t2,tp,q,u10,v10,sp,blh
time,,,,,,,,,
2021-03-30 18:59:59.999997,-8.75,13.25,300.694122,0.0,0.017289,2.862839,0.835739,100690.843750,546.046326
2021-03-30 20:00:00.000003,-8.75,13.25,300.440125,0.0,0.017369,2.234467,1.203201,100794.406250,512.461609
2021-03-30 21:00:00.000000,-8.75,13.25,300.186157,0.0,0.017384,1.724518,1.453690,100832.953125,441.784576
2021-03-30 21:59:59.999997,-8.75,13.25,299.549011,0.0,0.017056,1.663010,1.562439,100857.273438,390.854004
2021-03-30 23:00:00.000003,-8.75,13.25,299.419128,0.0,0.017062,1.371246,1.439163,100840.781250,297.748688


In [24]:
#Converter os dados unidades de cada data frame (parametro) unico.

data_df ['t2'] = data_df ['t2'] - 273.15
data_df ['tp'] = data_df ['tp']*1000
data_df ['q'] = data_df ['q']
data_df ['u10'] = data_df ['u10']
data_df ['v10'] = data_df ['v10']
data_df ['sp'] = data_df ['sp']/100
data_df ['blh'] = data_df ['blh']


In [26]:
# Selecionar os todos os dadose mudar os nomes
df_pr = pr.sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_t2m = t2m.sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_HR = HR['q'].sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_dir = u_wind['u10'].sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_speed = v_wind['v10'].sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_pressure = sp.sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))
df_blh = blh.sel(lat=(-8.82), lon=(13.22), method='nearest').sel(time=slice('2021-03-01', '2021-03-30'))


In [25]:
data_df

,lat,lon,t2,tp,q,u10,v10,sp,blh
time,,,,,,,,,
2021-03-01 00:00:00.000000,-8.75,13.25,25.987091,0.0,0.017407,0.826431,3.026978,1005.316345,446.928650
2021-03-01 00:59:59.999997,-8.75,13.25,25.817535,0.0,0.017420,0.461060,3.130447,1004.899841,462.804993
2021-03-01 02:00:00.000003,-8.75,13.25,25.416168,0.0,0.017444,0.009598,2.948959,1004.615784,476.581573
2021-03-01 03:00:00.000000,-8.75,13.25,25.586151,0.0,0.017453,-0.408035,2.615021,1004.624084,437.047516
2021-03-01 03:59:59.999997,-8.75,13.25,25.435699,0.0,0.017451,-0.770386,2.416183,1005.045715,381.725128
...,...,...,...,...,...,...,...,...,...
2021-03-30 18:59:59.999997,-8.75,13.25,27.544128,0.0,0.017289,2.862839,0.835739,1006.908447,546.046326
2021-03-30 20:00:00.000003,-8.75,13.25,27.290131,0.0,0.017369,2.234467,1.203201,1007.944092,512.461609
2021-03-30 21:00:00.000000,-8.75,13.25,27.036163,0.0,0.017384,1.724518,1.453690,1008.329529,441.784576


In [28]:
df_pr

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    tp       (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Surface Total precipitation
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [29]:
df_t2m

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    t2       (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [30]:
df_HR

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    q        (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Specific humidity on 1000 hPa
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [32]:
df_speed

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    v10      (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Surface 10 metre V wind component
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [33]:
df_pressure

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    sp       (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Surface Pressure
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [34]:
df_blh

<xarray.Dataset> Size: 9kB
Dimensions:  (time: 720)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2021-03-01 ... 2021-03-30T23:00:00.000003
    lat      float64 8B -8.75
    lon      float64 8B 13.25
Data variables:
    blh      (time) float32 3kB ...
Attributes:
    title:          ERA5 hourly Boundary layer height
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Sat Nov 02 18:18:07 HST 2024 : imported by GrADS Data Ser...

In [36]:
# função final que permite importar a direção do vento e velocidade em xarray como função xr.
final_dir = xr.apply_ufunc (wind_dir , df_dir , df_speed , dask='allowed', vectorize=True)
final_speed = xr.apply_ufunc (wind_speed , df_dir , df_speed , dask='allowed', vectorize=True)

ValueError: exact match required for all data variable names, but ['v10'] != ['u10']: {'v10', 'u10'} are not in both.

AttributeError: 'tuple' object has no attribute 'to_dataframe'

#Extrair dados temperatura da parte de Angola do periodo 2013 a 2022




In [ ]:
t2m_ang = t2m.sel(lat=slice(-19 , -3), lon=slice(11 , 25), time=slice('2022-01-01', '2022-12-31'))

In [ ]:
t2m_ang


<xarray.Dataset>
Dimensions:  (time: 8760, lat: 65, lon: 57)
Coordinates:
  * time     (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00.000003
  * lat      (lat) float64 -19.0 -18.75 -18.5 -18.25 ... -3.75 -3.5 -3.25 -3.0
  * lon      (lon) float64 11.0 11.25 11.5 11.75 12.0 ... 24.25 24.5 24.75 25.0
Data variables:
    t2       (time, lat, lon) float32 ...
Attributes:
    title:          ERA5 hourly Surface 2 metre temperature
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/datadoc/ecmwf_ERA5.php
    history:        Mon Jul 08 03:35:55 HST 2024 : imported by GrADS Data Ser...

In [ ]:
## Selecionar a região de interesse
Munhango_pi = t2m_ang.sel(lat=(-12.15740000), lon=(18.55522222), method=('nearest'))


In [ ]:
Munhango_pi

In [ ]:
#Dados de hora em hora para mês
montlhy_temp = Munhango_pi.resample(time='1M').mean()

In [ ]:
montlhy_temp

In [ ]:
##Unit conversion celsius data montlhy
mes_convert = montlhy_temp['t2'] - 273.15

In [ ]:
mes_convert

In [ ]:
#Passar dados de hora em hora para mensal
montlhy_tmin = Munhango_pi.resample(time='1M').min()

In [ ]:
montlhy_tmax = Munhango_pi.resample(time='1M').max()

In [ ]:
## Conveter unidades em graus celsius Tmin e Tmax
Tmin_mensal = montlhy_tmin['t2'] - 273.15


In [ ]:
## Conveter unidades em graus celsius Tmax
Tmax_mensal = montlhy_tmax['t2'] - 273.15

In [ ]:
#Make Hourly series from monthly series
import os
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt



In [ ]:
df_temp = pd.DataFrame(Munhango_pi)

In [ ]:
df_temp

In [ ]:
from dask.array.routines import average
averageData = df_temp.groupby([lambda x:x.month]).mean()

#average [['Tmin' ,'Tmax']]

In [ ]:
#Df
filename = ('Munhango_pi')
# Initialize dataframe to hold data.
df_temp = pd.DataFrame(filename)
print(len(temp),len(minT),len(maxT))
# Populate date and average temperature fields
# Change text date to datetime formate.
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
# Convert temperature from tenths of Celsius to Fahrenheit.
df_temp['avgTemp'] = [float (t2) - 273.15 for t2 in temps]
df_temp['minTemp'] = [float (t2) - 273.15 for t2 in minT]
df_temp['maxTemp'] = [float (t2) - 273.15 for t2 in maxT]
# Convert precipitation from mm/day to inch/day.
df_temp['prcp'] = [float(v)/25.54 for v in prcp]

# Show us what's at the top of the dataframe.
# This is a good way to check you got the
# data you expected!
print(df_temp.head())